# 지도 시각화

In [39]:
import pandas as pd
import folium
import numpy as np
import os
import bs4
import requests
from IPython.display import SVG

In [40]:
D=pd.read_excel('https://data.hossam.kr/D04/clinic.xlsx')
data=pd.DataFrame(D)

### 서울시 지도 다운로드

In [41]:
if os.path.exists("./SEOUL_SIG.html"):
    with open("./SEOUL_SIG.html","r",encoding='utf-8') as f:
        M=f.read()

### 데이터 확인

In [42]:
data.columns

Index(['기준일', '검체채취\n가능여부', '시도', '시군구', '의료기관명', '주소', '평일 운영시간', '토요일 운영시간',
       '일요일/공휴일\n운영시간', '대표 전화번호'],
      dtype='object')

In [43]:
data.drop(columns=["기준일",'대표 전화번호'],axis=1,inplace=True)

### 데이터 확인(Nan)

In [44]:
data.isna().sum()

검체채취\n가능여부       0
시도               0
시군구              0
의료기관명            0
주소               0
평일 운영시간          0
토요일 운영시간         0
일요일/공휴일\n운영시간    0
dtype: int64

In [45]:
#서울
Seoul=data[data["시도"]=="서울"]

In [46]:
# 이것도 필요없는 데이터
Seoul[Seoul.columns[0]].value_counts()

검체채취\n가능여부
○    71
Name: count, dtype: int64

In [47]:
Seoul.drop(columns=['검체채취\n가능여부'],inplace=True)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_33136\526171706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Seoul.drop(columns=['검체채취\n가능여부'],inplace=True)


In [48]:
Seoul.dtypes

시도               object
시군구              object
의료기관명            object
주소               object
평일 운영시간          object
토요일 운영시간         object
일요일/공휴일\n운영시간    object
dtype: object

### 구분 만들기

In [49]:
# 평일, 토요일, 일요일/공휴일
temp=Seoul.columns[-3:]

In [50]:
area=dict(zip(Seoul["시군구"].unique(),range(Seoul["시군구"].nunique())))

In [52]:
cols=['의료기관명','시군구_구분']

### 그전에!! 전처리(활동안되는 병원을 어떻게 구분할것인가)

In [53]:
Seoul["시군구_구분"]=Seoul["시군구"].apply(lambda x:area[x])

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_33136\4200122408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Seoul["시군구_구분"]=Seoul["시군구"].apply(lambda x:area[x])


### 결과 도출 데이터 프레임

In [55]:
result=pd.DataFrame(index=area.values(),data=area.keys())
result.index.rename("시군구_구분",inplace=True)
result.rename(columns={0:"시군구"},inplace=True)

In [16]:
for col in temp:
    result=pd.merge(result,
                    pd.DataFrame(Seoul[Seoul[col].str.find("~")!=-1][cols].groupby("시군구_구분").count()).rename(columns={"의료기관명":col}),
                    how="left",
                    left_index=True, right_index=True)

In [58]:
result.rename(columns=dict(zip(result.columns[1:],["평일","토요일","공휴일"])),inplace=True)

In [59]:
result

,시군구
시군구_구분,
0,강남구
1,강동구
2,강북구
3,강서구
4,관악구
5,광진구
6,구로구
7,금천구
8,노원구


In [18]:
result["Sum"]=result[result.columns[-3:]].sum(axis=1)

In [19]:
result

,시군구,평일,토요일,공휴일,Sum
시군구_구분,,,,,
0,강남구,3,3,2,8
1,강동구,4,3,1,8
2,강북구,1,1,1,3
3,강서구,1,1,1,3
4,관악구,2,2,1,5
5,광진구,2,2,1,5
6,구로구,3,2,1,6
7,금천구,2,2,2,6
8,노원구,3,1,1,5


In [20]:
result.isna().sum()

시군구    0
평일     0
토요일    0
공휴일    0
Sum    0
dtype: int64

In [21]:
result["Sum"].describe()

count    25.000000
mean      6.440000
std       2.142429
min       3.000000
25%       5.000000
50%       6.000000
75%       8.000000
max      11.000000
Name: Sum, dtype: float64

### 지도 매핑

SVG 파일 만들기..

한줄씩 추출 -> 맵 상의 색 하얀색으로 초기화

In [62]:
if os.path.exists("./SEOUL_SIG.html"):
    with open("./SEOUL_SIG.html","r",encoding='utf-8') as f:
        SIG=f.readlines()

In [63]:
pre_html=[]
for raw in SIG:
    if 'fill:' in raw:
        try:
            pre_html.append(raw.replace(raw[raw.index('#'):raw.index(';')],"#ffffff"))
        except:
            pre_html.append(raw)    
    else:
        pre_html.append(raw)

#### 색 규칙을 위한 처리

In [24]:
dsc=pd.DataFrame(result["Sum"].describe())
dsc

,Sum
count,25.000000
mean,6.440000
std,2.142429
min,3.000000
25%,5.000000
50%,6.000000
75%,8.000000
max,11.000000


In [25]:
#색 놀이
color_rules=dict(zip(range(5),['#F5F6CE','#F3F781','#F7FE2E',"#D7DF01",'#393B0B']))

In [26]:
#기준정하기
score_rules={
4:result[result["Sum"]<=dsc["Sum"].loc['25%']]["시군구"].values,
3:result[result["Sum"]<=dsc["Sum"].loc['50%']]["시군구"].values,
2:result[result["Sum"]<=dsc["Sum"].loc['75%']]["시군구"].values,
1:result[result["Sum"]>dsc["Sum"].loc['75%']]["시군구"].values,
0:result[result["Sum"]==dsc["Sum"].loc['max']]["시군구"].values}
#주의! 1을 먼저 매핑해야 0번이 의미있다.

#### id 값 처리

In [65]:
Msoup=bs4.BeautifulSoup(M)

In [66]:
id_dict={}
for x in Msoup.select(".TEXT"):
    id_dict[x.text]="#"+x["id"][1:]+" {\n"

In [67]:
id_dict

{'종로구': '#CD11110 {\n',
 '중구': '#CD11140 {\n',
 '용산구': '#CD11170 {\n',
 '성동구': '#CD11200 {\n',
 '광진구': '#CD11215 {\n',
 '동대문구': '#CD11230 {\n',
 '중랑구': '#CD11260 {\n',
 '성북구': '#CD11290 {\n',
 '강북구': '#CD11305 {\n',
 '도봉구': '#CD11320 {\n',
 '노원구': '#CD11350 {\n',
 '은평구': '#CD11380 {\n',
 '서대문구': '#CD11410 {\n',
 '마포구': '#CD11440 {\n',
 '양천구': '#CD11470 {\n',
 '강서구': '#CD11500 {\n',
 '구로구': '#CD11530 {\n',
 '금천구': '#CD11545 {\n',
 '영등포구': '#CD11560 {\n',
 '동작구': '#CD11590 {\n',
 '관악구': '#CD11620 {\n',
 '서초구': '#CD11650 {\n',
 '강남구': '#CD11680 {\n',
 '송파구': '#CD11710 {\n',
 '강동구': '#CD11740 {\n'}

In [29]:
pre_html

['<style>\n',
 '.OUTLINE {\n',
 '    stroke-linejoin:round;\n',
 '    stroke: #ffffff;\n',
 '    stroke-width: 2;\n',
 '}\n',
 '#CD11110 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11140 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11170 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11200 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11215 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11230 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11260 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11290 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11305 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11320 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11350 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11380 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11410 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11440 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11470 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11500 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11530 {\n',
 '    fill: #ffffff;\n',
 '}\n',
 '#CD11545 {\n',
 '    f

In [30]:
id_dict

{'종로구': '#CD11110 {\n',
 '중구': '#CD11140 {\n',
 '용산구': '#CD11170 {\n',
 '성동구': '#CD11200 {\n',
 '광진구': '#CD11215 {\n',
 '동대문구': '#CD11230 {\n',
 '중랑구': '#CD11260 {\n',
 '성북구': '#CD11290 {\n',
 '강북구': '#CD11305 {\n',
 '도봉구': '#CD11320 {\n',
 '노원구': '#CD11350 {\n',
 '은평구': '#CD11380 {\n',
 '서대문구': '#CD11410 {\n',
 '마포구': '#CD11440 {\n',
 '양천구': '#CD11470 {\n',
 '강서구': '#CD11500 {\n',
 '구로구': '#CD11530 {\n',
 '금천구': '#CD11545 {\n',
 '영등포구': '#CD11560 {\n',
 '동작구': '#CD11590 {\n',
 '관악구': '#CD11620 {\n',
 '서초구': '#CD11650 {\n',
 '강남구': '#CD11680 {\n',
 '송파구': '#CD11710 {\n',
 '강동구': '#CD11740 {\n'}

In [31]:
score_rules[3]

array(['강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '마포구',
       '서대문구', '서초구', '성동구', '성북구', '용산구'], dtype=object)

In [32]:
color_rules

{0: '#F5F6CE', 1: '#F3F781', 2: '#F7FE2E', 3: '#D7DF01', 4: '#393B0B'}

In [33]:
for num in [1,0,2,3,4]: #매핑 순서
    for area in score_rules[num]: #지역
        for i in range(len(pre_html)):
            if pre_html[i]==id_dict[area]:
                pre_html[i+1]=f'    fill: {color_rules[num]};\n' #이쁘게 바꿨겠죠?

In [34]:
with open("./MySeoul.html","w") as f:
    for html in pre_html:
        f.write(html)

In [35]:
color_rules[4]

'#393B0B'

In [36]:
for i in range(len(pre_html)):
            if pre_html[i]==id_dict["강남구"]:
                print(pre_html[i])

#CD11680 {

